<a href="https://colab.research.google.com/github/LES4975/Album_Art_Generator_Project/blob/main/test_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sdxl_lightning_test.py
# Google Colab용 SDXL-Lightning 테스트 앱

# =============================================================================
# 1. 라이브러리 설치 (매우 간단!)
# =============================================================================

# SDXL Lightning는 최신 diffusers만 필요
!pip install diffusers accelerate gradio torch torchvision transformers
print("✅ 라이브러리 설치 완료!")


✅ 라이브러리 설치 완료!


In [ ]:

# =============================================================================
# 2. 라이브러리 import
# =============================================================================

import gradio as gr
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from PIL import Image
import time
import gc


In [ ]:
# =============================================================================
# 3. SDXL-Turbo 앱 클래스
# =============================================================================

class SDXLLightningApp:
    """SDXL Lightning Gradio 앱 클래스"""

    def __init__(self, base_model="stabilityai/stable-diffusion-xl-base-1.0",
                 lightning_repo="ByteDance/SDXL-Lightning"):
        """
        초기화

        Args:
            base_model: 기본 SDXL 모델
            lightning_repo: Lightning LoRA 리포지토리
        """
        self.base_model = base_model
        self.lightning_repo = lightning_repo
        self.pipeline = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"🔧 SDXL Lightning 초기화 중... (디바이스: {self.device})")
        self._load_pipeline()

    def _load_pipeline(self):
        """SDXL Lightning 파이프라인 로드"""
        try:
            print(f"📦 기본 SDXL 모델 로딩 중: {self.base_model}")

            # 기본 SDXL 파이프라인 로드
            self.pipeline = StableDiffusionXLPipeline.from_pretrained(
                self.base_model,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                variant="fp16" if self.device == "cuda" else None,
                use_safetensors=True
            )

            print(f"📦 SDXL Lightning LoRA 로딩 중...")

            # Lightning 4-step LoRA 다운로드 및 로드
            lightning_lora_path = hf_hub_download(
                repo_id=self.lightning_repo,
                filename="sdxl_lightning_4step_lora.safetensors"
            )

            # LoRA 어댑터 로드
            self.pipeline.load_lora_weights(lightning_lora_path)

            # Lightning용 스케줄러 설정
            self.pipeline.scheduler = EulerDiscreteScheduler.from_config(
                self.pipeline.scheduler.config,
                timestep_spacing="trailing"
            )

            # GPU 최적화
            if self.device == "cuda":
                self.pipeline = self.pipeline.to("cuda")
                self.pipeline.enable_attention_slicing()
                print("✅ GPU 모드 활성화")
            else:
                print("⚠️ CPU 모드 (속도 느림)")

            print("✅ SDXL Lightning 파이프라인 로드 완료!")

        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            self.pipeline = None

    def generate_image(self, prompt, negative_prompt="", num_steps=4, guidance_scale=0.0,
                      width=1024, height=1024, seed=-1):
        """
        이미지 생성 (SDXL Lightning 최적화)

        Args:
            prompt: 생성 프롬프트
            negative_prompt: 네거티브 프롬프트
            num_steps: 추론 스텝 (2, 4, 8 권장)
            guidance_scale: 가이던스 스케일 (Lightning은 0.0 권장)
            width, height: 이미지 크기 (1024x1024 권장)
            seed: 시드값

        Returns:
            generated_image: PIL Image 객체
            generation_info: 생성 정보
        """

        if self.pipeline is None:
            return None, "❌ 모델이 로드되지 않았습니다"

        if not prompt.strip():
            return None, "❌ 프롬프트를 입력해주세요"

        try:
            print(f"⚡ SDXL Lightning 이미지 생성 시작...")
            print(f"📝 프롬프트: {prompt}")

            # 시드 설정
            if seed != -1:
                torch.manual_seed(seed)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed(seed)

            # 시작 시간 기록
            start_time = time.time()

            # 이미지 생성 (SDXL Lightning 최적화 파라미터)
            result = self.pipeline(
                prompt=prompt,
                negative_prompt=negative_prompt if negative_prompt.strip() else None,
                num_inference_steps=num_steps,  # Lightning: 2, 4, 8 스텝 권장
                guidance_scale=guidance_scale,   # Lightning: 0.0 권장
                width=width,
                height=height
            )

            # 생성 시간 계산
            generation_time = time.time() - start_time

            # 생성 정보
            generation_info = f"""
⚡ SDXL Lightning 생성 완료!
🚀 생성 시간: {generation_time:.1f}초
📏 크기: {width}x{height}
🎯 스텝: {num_steps} (Lightning 모드)
🎛️ 가이던스: {guidance_scale}
🎲 시드: {seed if seed != -1 else '랜덤'}
💎 품질: 고해상도 (Lightning 최적화)
            """.strip()

            print(generation_info)

            # GPU 메모리 정리
            if self.device == "cuda":
                torch.cuda.empty_cache()
                gc.collect()

            return result.images[0], generation_info

        except Exception as e:
            error_msg = f"❌ 이미지 생성 실패: {str(e)}"
            print(error_msg)
            return None, error_msg


In [ ]:
# =============================================================================
# 4. Gradio 인터페이스 생성
# =============================================================================

def create_gradio_interface():
    """SDXL Lightning Gradio 인터페이스 생성"""

    # SDXL Lightning 앱 초기화
    app = SDXLLightningApp()

    # Gradio 인터페이스 정의
    with gr.Blocks(title="⚡ SDXL Lightning 이미지 생성기", theme=gr.themes.Soft()) as demo:

        # 헤더
        gr.Markdown("""
        # ⚡ SDXL Lightning 이미지 생성기

        고품질 + 고속도! 2-8 스텝으로 1024x1024 고해상도 이미지를 생성하세요.
        **Turbo보다 품질 우수, 일반 SDXL보다 10배 빠름!**
        """)

        with gr.Row():
            # 입력 섹션
            with gr.Column(scale=1):
                gr.Markdown("### 📝 입력")

                prompt_input = gr.Textbox(
                    label="🎯 프롬프트",
                    placeholder="예: professional album cover design, artistic, vibrant colors, detailed",
                    lines=3,
                    value="beautiful album cover design, artistic, professional, vibrant colors, high quality, detailed"
                )

                negative_prompt_input = gr.Textbox(
                    label="🚫 네거티브 프롬프트",
                    placeholder="예: blurry, low quality, text, watermark",
                    lines=2,
                    value="blurry, low quality, ugly, deformed, text, watermark, signature, logo"
                )

                with gr.Row():
                    width_input = gr.Slider(
                        label="📏 너비",
                        minimum=512,
                        maximum=1536,
                        step=64,
                        value=1024
                    )
                    height_input = gr.Slider(
                        label="📏 높이",
                        minimum=512,
                        maximum=1536,
                        step=64,
                        value=1024
                    )

                with gr.Row():
                    steps_input = gr.Slider(
                        label="🔄 생성 스텝",
                        minimum=2,
                        maximum=8,
                        step=2,
                        value=4,
                        info="Lightning 모드: 2, 4, 8 스텝 권장"
                    )
                    guidance_input = gr.Slider(
                        label="🎛️ 가이던스",
                        minimum=0.0,
                        maximum=2.0,
                        step=0.1,
                        value=0.0,
                        info="Lightning 모드: 0.0 권장"
                    )

                seed_input = gr.Number(
                    label="🎲 시드 (-1=랜덤)",
                    value=-1,
                    precision=0
                )

                generate_btn = gr.Button(
                    "⚡ Lightning 생성",
                    variant="primary",
                    size="lg"
                )

            # 출력 섹션
            with gr.Column(scale=2):
                gr.Markdown("### 🖼️ 결과")

                info_output = gr.Textbox(
                    label="📋 상태",
                    value="프롬프트를 입력하고 생성 버튼을 눌러주세요",
                    lines=4,
                    interactive=False
                )

                image_output = gr.Image(
                    label="⚡ 생성된 이미지 (고해상도)",
                    type="pil",
                    height=500
                )

        # 앨범 커버 프롬프트 예시 및 비교
        with gr.Row():
            gr.Markdown("""
            ### 🎵 앨범 커버 프롬프트 예시 (고품질 Lightning용)

            **클래식/아트:**
            - `elegant classical album cover, minimalist design, golden ratio, sophisticated typography, art nouveau style`
            - `vintage jazz album artwork, warm sepia tones, vinyl record aesthetic, retro elegance`

            **팝/모던:**
            - `modern pop album cover, vibrant gradients, geometric shapes, contemporary design, neon accents`
            - `colorful abstract album art, fluid dynamics, digital art style, trending on artstation`

            **록/메탈:**
            - `dark rock album cover, dramatic lighting, bold typography, grunge aesthetic, high contrast`
            - `metal music artwork, epic fantasy art, detailed illustration, powerful imagery`

            **일렉트로닉:**
            - `futuristic electronic music cover, cyberpunk aesthetic, neon lights, digital art, synthwave style`
            - `abstract electronic album art, particle effects, holographic textures, sci-fi design`

            ### ⚡ SDXL Lightning 특징
            - **고품질**: Turbo 대비 현저히 우수한 품질
            - **고해상도**: 1024x1024 네이티브 지원 (Turbo는 512x512만)
            - **균형**: 속도와 품질의 완벽한 밸런스
            - **유연성**: 2-8 스텝으로 품질 조절 가능
            - **호환성**: LoRA 및 다른 모델과 호환성 우수

            ### 📊 성능 비교
            | 모델 | 해상도 | 스텝 | 품질 | 속도 |
            |------|--------|------|------|------|
            | SDXL Lightning | 1024x1024 | 2-8 | ⭐⭐⭐⭐⭐ | ⚡⚡⚡⚡ |
            | SDXL Turbo | 512x512 | 1-4 | ⭐⭐⭐ | ⚡⚡⚡⚡⚡ |
            | SDXL 1.0 | 1024x1024 | 20-50 | ⭐⭐⭐⭐⭐ | ⚡⚡ |
            """)

        # 이벤트 연결
        generate_btn.click(
            fn=app.generate_image,
            inputs=[
                prompt_input,
                negative_prompt_input,
                steps_input,
                guidance_input,
                width_input,
                height_input,
                seed_input
            ],
            outputs=[image_output, info_output]
        )

    return demo

In [ ]:

# =============================================================================
# 5. 앱 실행
# =============================================================================

def main():
    """메인 실행 함수"""
    print("⚡ SDXL Lightning 테스트 앱 시작!")
    print("=" * 50)

    # GPU 확인
    if torch.cuda.is_available():
        print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name()}")
        print(f"📊 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    else:
        print("⚠️ GPU 사용 불가 - CPU 모드 (속도 매우 느림)")

    # Gradio 앱 생성 및 실행
    demo = create_gradio_interface()

    # 앱 실행
    demo.launch(
        share=True,  # 공유 링크 생성
        debug=True,
        server_name="0.0.0.0",
        server_port=7860
    )


# 직접 실행
if __name__ == "__main__":
    main()


⚡ SDXL Lightning 테스트 앱 시작!
✅ GPU 사용 가능: Tesla T4
📊 GPU 메모리: 15.8GB
🔧 SDXL Lightning 초기화 중... (디바이스: cuda)
📦 기본 SDXL 모델 로딩 중: stabilityai/stable-diffusion-xl-base-1.0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

⚡ SDXL Lightning 생성 완료!
🚀 생성 시간: 6.6초
📏 크기: 1024x1024
🎯 스텝: 4 (Lightning 모드)
🎛️ 가이던스: 0
🎲 시드: 랜덤
💎 품질: 고해상도 (Lightning 최적화)
⚡ SDXL Lightning 이미지 생성 시작...
📝 프롬프트: dark rock album cover, dramatic lighting, bold typography, grunge aesthetic, high contrast


  0%|          | 0/4 [00:00<?, ?it/s]

⚡ SDXL Lightning 생성 완료!
🚀 생성 시간: 4.4초
📏 크기: 1024x1024
🎯 스텝: 4 (Lightning 모드)
🎛️ 가이던스: 0
🎲 시드: 랜덤
💎 품질: 고해상도 (Lightning 최적화)
⚡ SDXL Lightning 이미지 생성 시작...
📝 프롬프트: dark rock album cover, dramatic lighting, grunge aesthetic, high contrast


  0%|          | 0/4 [00:00<?, ?it/s]

⚡ SDXL Lightning 생성 완료!
🚀 생성 시간: 4.5초
📏 크기: 1024x1024
🎯 스텝: 4 (Lightning 모드)
🎛️ 가이던스: 0
🎲 시드: 랜덤
💎 품질: 고해상도 (Lightning 최적화)
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://efd5ee3ed88d1f0070.gradio.live


In [ ]:
# =============================================================================
# 6. 빠른 테스트 함수
# =============================================================================

def quick_test():
    """빠른 테스트 함수"""
    print("🧪 SDXL Lightning 빠른 테스트...")

    app = SDXLLightningApp()

    if app.pipeline is None:
        print("❌ 모델 로드 실패")
        return

    # 테스트 이미지 생성 (4 스텝으로 고품질!)
    test_prompt = "beautiful album cover design, artistic, vibrant colors, professional, detailed"
    image, info = app.generate_image(test_prompt, num_steps=4)

    if image:
        print("✅ 테스트 성공!")
        image.show()  # Colab에서 이미지 표시
        print(info)
    else:
        print(f"❌ 테스트 실패: {info}")

# 빠른 테스트 실행 (주석 해제하여 사용)
# quick_test()